In [235]:
import pandas as pd

In [236]:
file = pd.read_csv('D:\\datasets\\Star3642_balanced.csv')
df = file.copy()
df.sample(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
3201,8.79,0.21,1.39,1.161,K1III,10.401096,0
424,6.69,7.47,0.81,0.999,K0III,16.056602,0
892,4.34,19.34,0.76,0.871,G5III...,15.772284,0
1992,8.50,6.69,0.80,0.447,F3V,17.627130,1
1959,8.83,5.64,1.05,0.468,F3/F5V,17.586395,1
2144,8.02,4.40,0.86,0.970,G4III,16.237263,0
590,8.42,4.14,1.05,1.005,G8III,16.505001,0
491,8.75,2.49,0.83,1.252,K2IIICN...,15.730997,0
2052,12.04,27.77,3.55,1.453,M2V:,24.257880,1
640,8.21,1.97,0.98,1.557,K5III,14.682331,0


In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642 entries, 0 to 3641
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Vmag         3642 non-null   float64
 1   Plx          3642 non-null   float64
 2   e_Plx        3642 non-null   float64
 3   B-V          3642 non-null   float64
 4   SpType       3642 non-null   object 
 5   Amag         3642 non-null   float64
 6   TargetClass  3642 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 199.3+ KB


In [238]:
df['SpType'].value_counts(ascending=False)

K0III          293
K1III          200
K2III          156
G8III          135
F5V            125
              ... 
B9.5IV           1
B0Ibp            1
M0V              1
B1Vv SB          1
M0/M1IIICNp      1
Name: SpType, Length: 584, dtype: int64

In [239]:
len(list(set(df['SpType'])))

584

In [240]:
from sklearn.preprocessing import LabelEncoder , StandardScaler
encoder = LabelEncoder()

In [241]:
label = df['TargetClass']
sptype = df['SpType']
df= df.drop(['SpType','TargetClass'],axis=1)
df.head()

,Vmag,Plx,e_Plx,B-V,Amag
0,5.99,13.73,0.58,1.318,16.678352
1,8.70,2.31,1.29,-0.045,15.518060
2,5.77,5.50,1.03,0.855,14.471813
3,6.72,5.26,0.74,-0.015,15.324928
4,8.76,13.44,1.16,0.584,19.401997


In [242]:
encoded_sptype = pd.DataFrame(encoder.fit_transform(sptype),columns=['sptype'])
new_df = pd.concat([df,encoded_sptype],axis=1)
new_df.sample(10)

,Vmag,Plx,e_Plx,B-V,Amag,sptype
370,8.48,0.95,1.03,1.461,13.368618,474
2763,9.09,5.75,1.22,0.379,17.888340,226
309,6.85,8.21,0.91,1.028,16.421715,389
2595,7.94,9.03,1.29,0.392,17.718440,224
451,7.04,2.90,0.84,1.428,14.351990,515
2468,9.15,5.75,1.40,0.537,17.948340,244
1866,5.52,57.63,0.64,0.551,19.323244,283
1966,9.36,19.65,1.47,0.928,20.826813,496
1042,8.68,17.32,1.19,0.638,19.872740,320
3228,8.34,4.09,1.16,1.140,16.398617,462


In [243]:
scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(new_df),columns=new_df.columns)
scaled_df.describe()

,Vmag,Plx,e_Plx,B-V,Amag,sptype
count,3.642000e+03,3.642000e+03,3.642000e+03,3.642000e+03,3.642000e+03,3.642000e+03
mean,-7.480745e-16,1.013589e-17,-6.209948e-17,1.048646e-17,1.047487e-15,5.423083e-17
std,1.000137e+00,1.000137e+00,1.000137e+00,1.000137e+00,1.000137e+00,1.000137e+00
min,-5.729871e+00,-1.210003e+00,-7.681151e-01,-2.343073e+00,-5.604798e+00,-2.450110e+00
25%,-5.606444e-01,-4.081516e-01,-3.658854e-01,-7.782791e-01,-5.260197e-01,-7.221359e-01
50%,2.012936e-01,-2.227865e-01,-1.484639e-01,9.143281e-02,-1.437642e-02,2.055878e-01
75%,7.017823e-01,8.923081e-02,1.015708e-01,7.075026e-01,6.600385e-01,8.217097e-01
max,3.263986e+00,2.286985e+01,3.164943e+01,3.900072e+00,5.281210e+00,1.678615e+00


In [244]:
from sklearn.model_selection import train_test_split
xtr, xts , ytr , yts = train_test_split(scaled_df,label,test_size=0.2,random_state=42)
print(len(xtr))
print(len(ytr))
print(len(xts))
print(len(yts))

2913
2913
729
729


In [245]:
from sklearn.ensemble import ExtraTreesClassifier
alg = ExtraTreesClassifier().fit(xtr,ytr)
alg.score(xtr,ytr)*100

100.0

In [246]:
alg.score(xts,yts)*100

90.6721536351166

In [247]:
alg.score(scaled_df,label)*100

98.13289401427787

In [248]:
bad_result = 0
for i in range(10):
    xtr, xts , ytr , yts = train_test_split(scaled_df,label,test_size=0.2,shuffle=True)
    alg = ExtraTreesClassifier().fit(xtr,ytr)
    test = (alg.score(xts,yts)*100) - (alg.score(scaled_df,label)*100)
    if test>15 :
        bad_result+=1
bad_result

0